Using LSTM to generate sin function. 

Set up: 

- Training : using 50 numbers as X to generate the 51st number as y and so on. 
- Test : test on 20 numbers

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline

In [2]:
# defining model: 
class LSTM(nn.Module):
    def __init__(self,input_dim,out_dim,hidden_dim):
      super(LSTM,self).__init__()
      self.input_dim = input_dim
      self.hidden_dim = hidden_dim
      self.out_dim = out_dim
      self.LSTM = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional = False, )
      self.ln = nn.Linear(1*hidden_dim, self.out_dim)
      self.ln2 = nn.Linear(self.out_dim, self.out_dim)
    def forward (self, inp):
      output,_ = self.LSTM(inp)
      x = torch.tanh(self.ln(output))
      x = self.ln2(x)
      return x

In [3]:
model = LSTM(50,1,20)

In [4]:
model.double().to('cuda')

LSTM(
  (LSTM): LSTM(50, 20, batch_first=True)
  (ln): Linear(in_features=20, out_features=1, bias=True)
  (ln2): Linear(in_features=1, out_features=1, bias=True)
)

In [11]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.08)

In [6]:
#generate training and testing data: 
def train_function(X, function ='sin'):
  if function == 'cos':
    return np.cos(X)
  else: 
    return np.sin(X)

def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
                end_ix = i + n_steps
                if end_ix > len(sequence)-1:
                        break
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
        # X = np.array(X).astype(np.float32)
        # X = torch.from_numpy(X)
        # X = X.unsqueeze(dim=0)
        # y = np.array(y).astype(np.float32)
        # y = torch.from_numpy(y)
        X = torch.tensor(X)
        X = X.unsqueeze(dim=0)
        y = torch.tensor(y).reshape(-1,1)
        return X,y

#define input sequence: 
xaxis = np.arange(-100*np.pi, 100*np.pi, 0.1)
train_seq = train_function(xaxis)
n_steps = 50
X, y = split_sequence(train_seq, n_steps)



In [7]:
X.shape, y.shape

(torch.Size([1, 6234, 50]), torch.Size([6234, 1]))

In [12]:
#training with 1000 epochs: 

epochs = 200

for i in range(epochs):
  optimizer.zero_grad()
  y_pred = model(X.to('cuda'))
  single_loss = loss_function(y_pred[0], y.to('cuda'))
  single_loss.backward()
  optimizer.step()
  
  if i%25 == 1:
      print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.41483101
epoch:  26 loss: 0.00497311
epoch:  51 loss: 0.00100262
epoch:  76 loss: 0.00041769
epoch: 101 loss: 0.00024795
epoch: 126 loss: 0.00014392
epoch: 151 loss: 0.00009798
epoch: 176 loss: 0.00007921
epoch: 199 loss: 0.0000553740
